## **<font color="red">Artifacts</font>**
- **Definition:** An Artifact is essentially a piece of binary data (like the content of a file) identified by a unique `filename` string within aspecific scope(session or user). Each time you save an artifact with the same filename, a new version is created.
- **Representation:** Artifacts are consistently represented using the standard `google.genai.types.Part` object. The core data is typically stored within an inline data structure of the `Part` (accessed via `inline_data`), which itself contains:
  - `data`: The raw binary content as bytes.
  - `mime_type`: A string indicating the type of the data(e.g., `"image/png"`, `"applicaiton/pdf`"). This is essential for correctly interpreting the data later.
- **Persistence & Management:** Artifacts are not stored directly within the agent or session state. Their storage and retrieval are managed by a dedicated **Artifact Service** (an implementation of `BaseArtifactService`, defined in `google.adk.artifacts`. ):
  - An in-memory service for testing or temporary storage (e.g., `InMemoryArtifactService`, defined in `google.adk.artifacts.in_memory_artifact_service.py`).
  - A service for persistent storage using GCP (e.g, `GcsArtifactService` defined in `google.adk.artifacts.gcs_artifact_service.py`).

### **Why use Artifacts**
While session `state` is suitable for storing small pieces of configuration or conversational context (like strings, numbers, booleans, or small dictionaries/lists), Artifacts are designed for scenarios involving binary or large data:
1. **Handling Non-Textual Data:** Easily store and retrieve _images_, _audio clips_, _video snippets_, _PDFs_, _spreadsheets_, or any other file format relevant to your agent's function.
2. **Persisting Large Data:** Session state is generally not optimized for storing large amounts of data. Artifacts provide a dedicated mechanism for persisting larger blobs without cluttering the session state.
3. **User File Management:** Provide capabilities for users to upload files (which can be saved as artifacts) and retrieve or download files generated by the agent (loaded from artifacts).
4. **Sharing Outputs:** Enable tools or agents to generate binary outputs (like a PDF report or a generated image) that can be saved via `save_artifact` and later accessed by other parts of the application or even in subsequent sessions (if using user namespacing).
5. **Caching Binary Data:** Store the results of computationally expensive operations that produce binary data (e.g., rendering a complex chart image) as artifacts to avoid regenerating them on subsequent requests.
In essence, whenever your agent needs to work with file-like binary data that needs to be persisted, versioned, or shared, Artifacts managed by an `ArtifactService` are the appropriate mechanism within ADK.

### **Common Use Cases**
1. **Generated Report/Files:** A tool or agent generates a report (e.g., a _PDF analysis_, a _CSV data export_, an _image chart_).
2. **Handling User Uploads:** A user uploads a file (e.g., an _image for analysis_, a _document for summarization_) through a front-end interface.
3. **Storing Intermediate Binary Results:** An agent performs a complex multi-step process where one step generates intermediate binary data (e.g., audio synthesis, simulation results).
4. **Persistent User Data:** Storing user-specific configuration or data that isn't a simple key-value state.
5. **Caching Generated Binary Content:** An agent frequently generates the same binary output based on certain inputs (e.g., a company logo image, a standard audio greeting).

In [1]:
# ============================================================
# ADK Artifact Demo (CORRECT FOR YOUR INSTALLED VERSION)
# ============================================================

import os
import asyncio

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
import google.genai.types as types
from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

MODEL = "gemini-2.5-flash"
APP_NAME = "artifact_demo_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

# ============================================================
# TOOL 1: CREATE ARTIFACT
# ============================================================

async def create_report(topic: str, tool_context=None) -> str:

    report_content = f"""
===== REPORT =====
Topic: {topic}

This is a generated report about {topic}.
{tool_context}

==================
"""

    artifact_name = f"{topic}_report.txt"

    # Wrap bytes inside types.Part
    artifact_part = types.Part.from_bytes(
        data=report_content.encode("utf-8"),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=artifact_name,
        artifact=artifact_part
    )

    return f"Report saved as '{artifact_name}' (version {version})"

create_report_tool = FunctionTool(func=create_report)

# ============================================================
# TOOL 2: READ ARTIFACT
# ============================================================

async def read_report(artifact_name: str, tool_context=None) -> str:

    artifact_part = await tool_context.load_artifact(
        filename=artifact_name
    )

    if not artifact_part:
        return "Artifact not found."

    if artifact_part.inline_data:
        return artifact_part.inline_data.data.decode("utf-8")

    return "Artifact exists but has no binary content."

read_report_tool = FunctionTool(func=read_report)

# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="ArtifactAgent",
    model=MODEL,
    instruction="""
You are an assistant that:
- Generates reports and stores them as artifacts.
- Reads previously stored artifacts.

Always use tools when needed.
""",
    tools=[create_report_tool, read_report_tool]
)

# ============================================================
# RUNNER
# ============================================================

async def run_chat():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    print("Artifact Demo Started")
    print("Try:")
    print(" - Generate a report about AI")
    print(" - Read AI_report.txt")
    print("Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit", ""]:
            break

        content = types.Content(
            role="user",
            parts=[types.Part(text=user_input)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("Assistant:", event.content.parts[0].text)

# ============================================================
# ENTRY POINT
# ============================================================

if __name__ == "__main__":
    # asyncio.run(run_chat())
    await run_chat()


Artifact Demo Started
Try:
 - Generate a report about AI
 - Read AI_report.txt
Type 'exit' to quit.



You:  Generate a report about AI


D:\Agent-Development-Kit\venv\Lib\site-packages\google\adk\flows\llm_flows\base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


Assistant: I have generated a report about AI. It is saved as 'AI_report.txt' (version 0).


You:  Read AI_report.txt


Assistant: Here is the content of 'AI_report.txt':

===== REPORT =====
Topic: AI

This is a generated report about AI.



You:  


## **<font color="green">Core Concepts of Artifacts</font>**
Artifacts involves grasping a few key components: the service that manages them, the data structure used to hold them, and how they are identified and versioned.
### **<font color="blue">Artifact Service (`BaseArtifactService`)</font>**
- **Role:** The central component responsible for the actual storage and retrieval logic for artifacts. It defines _how_ and _where_ artifacts are persisted.
- **Interface:** Defined by the abstract base class `BaseArtifactService`.
  - `Save Artifact`: Stores the artifact data and returns its assigned version number.
  - `Load Artifact`: Retrieves a specific version (or the latest) of an artifact.
  - `List Artifact Keys`: Lists the unique filenames of artifacts within a given scope.
  - `Delete Artifact`: Remove a specific version (or the latest) of an artifact.
  - `List Versions`: Lists all available version numbers for a specific artifact filename.
- **Configurable:** You provide and instance(object) of an artifact service (e.g., `InMemoryArtifactService`, `GcsArtifactService`) when initializing the `Runner`. The `Runner` then makes this service available to agents and tools via the `InvocationContext`


In [2]:
# ============================================================
# ADK Artifact Service Full Demo
# Demonstrates:
# - Save Artifact
# - Load Artifact
# - List Artifact Keys
# - List Versions
# - Delete Artifact
# ============================================================

import os
import asyncio

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
import google.genai.types as types

from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

MODEL = "gemini-2.0-flash"
APP_NAME = "artifact_service_demo"
USER_ID = "user_1"
SESSION_ID = "session_001"

# ============================================================
# TOOL 1: SAVE ARTIFACT
# ============================================================

async def save_artifact_tool(filename: str, content: str, tool_context=None) -> str:

    artifact_part = types.Part.from_bytes(
        data=content.encode("utf-8"),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=artifact_part
    )

    return f"Artifact '{filename}' saved successfully (version {version})."


# ============================================================
# TOOL 2: LOAD ARTIFACT
# ============================================================

async def load_artifact_tool(filename: str, tool_context=None) -> str:

    artifact_part = await tool_context.load_artifact(filename=filename)

    if not artifact_part:
        return "Artifact not found."

    if artifact_part.inline_data:
        return artifact_part.inline_data.data.decode("utf-8")

    return "Artifact exists but contains no readable data."


# ============================================================
# TOOL 3: LIST ARTIFACT KEYS
# ============================================================

async def list_artifacts_tool(tool_context=None) -> str:

    keys = await tool_context.list_artifacts()

    if not keys:
        return "No artifacts found."

    return "Available Artifacts:\n" + "\n".join(keys)


# ============================================================
# TOOL 4: LIST VERSIONS
# ============================================================

async def list_versions_tool(filename: str, tool_context=None) -> str:

    versions = await tool_context.list_artifact_versions(filename=filename)

    if not versions:
        return "No versions found for this artifact."

    return f"Versions for '{filename}': {versions}"


# ============================================================
# TOOL 5: DELETE ARTIFACT
# ============================================================

async def delete_artifact_tool(filename: str, tool_context=None) -> str:

    await tool_context.delete_artifact(filename=filename)

    return f"Artifact '{filename}' deleted successfully."


# ============================================================
# WRAP TOOLS
# ============================================================

tools = [
    FunctionTool(func=save_artifact_tool),
    FunctionTool(func=load_artifact_tool),
    FunctionTool(func=list_artifacts_tool),
    FunctionTool(func=list_versions_tool),
    FunctionTool(func=delete_artifact_tool),
]

# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="ArtifactUserAgent",
    model=MODEL,
    instruction="""
You are an assistant that manages artifacts.

You can:
- Save artifacts
- Load artifacts
- List artifacts
- List versions
- Delete artifacts

Always use tools when performing artifact operations.
""",
    tools=tools
)

# ============================================================
# RUNNER SETUP WITH ARTIFACT SERVICE
# ============================================================

async def run_demo():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    print("=" * 60)
    print("Artifact Service Demo Started")
    print("=" * 60)
    print("Examples:")
    print(" - Save file1.txt with content Hello World")
    print(" - Load file1.txt")
    print(" - List artifacts")
    print(" - List versions of file1.txt")
    print(" - Delete file1.txt")
    print("Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit", ""]:
            break

        content = types.Content(
            role="user",
            parts=[types.Part(text=user_input)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("Assistant:", event.content.parts[0].text)


# ============================================================
# ENTRY POINT
# ============================================================

if __name__ == "__main__":
    # asyncio.run(run_demo())
    await run_demo()


Artifact Service Demo Started
Examples:
 - Save file1.txt with content Hello World
 - Load file1.txt
 - List artifacts
 - List versions of file1.txt
 - Delete file1.txt
Type 'exit' to quit.



You:  


### **<font color="blue">Artifact Data</font>**
- **Standard Representation:** Artifact content is universally represented using the `google.genai.types.Part` object, the same structure used for parts of LLM messages.
- **Key Attribute(`inline_data`):** For artifacts, the most relevant attribute is `inline_data`, which is a `google.genai.types.Blob` object containing:
  - **`data` (bytes):** A raw binary content of the artifact.
  - **`mime_type` (str):** A standard MIME type string(e.g., `application/pdf`, `image/png`, `audio/mpeg`) describing the nature of the binary data. This is crucial for correct interpretation when loading the artifact.

In [3]:
# ============================================================
# ADK Artifact Data Demo (Binary + MIME Type Example)
# Demonstrates:
# - types.Part
# - types.Blob (inline_data)
# - MIME type usage
# - Save binary artifact
# - Load and inspect MIME type
# ============================================================

import os
import asyncio

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
import google.genai.types as types

from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

MODEL = "gemini-2.0-flash"
APP_NAME = "artifact_data_demo"
USER_ID = "user_1"
SESSION_ID = "session_001"

# ============================================================
# TOOL 1: SAVE PDF ARTIFACT (Binary Example)
# ============================================================

async def save_pdf_tool(filename: str, tool_context=None) -> str:
    """
    Creates a mock PDF binary artifact and saves it.
    """

    # Mock PDF binary data
    pdf_bytes = b"%PDF-1.4\n%Mock PDF File for ADK Demo\n1 0 obj\n<< /Type /Catalog >>\nendobj\n"

    pdf_mime_type = "application/pdf"

    # Create Part using inline_data + Blob
    pdf_artifact = types.Part(
        inline_data=types.Blob(
            data=pdf_bytes,
            mime_type=pdf_mime_type
        )
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=pdf_artifact
    )

    return f"PDF artifact '{filename}' saved successfully (version {version})."


# ============================================================
# TOOL 2: LOAD ARTIFACT AND INSPECT METADATA
# ============================================================

async def load_artifact_tool(filename: str, tool_context=None) -> str:
    """
    Loads artifact and displays MIME type and size.
    """

    artifact_part = await tool_context.load_artifact(filename=filename)

    if not artifact_part:
        return "Artifact not found."

    if artifact_part.inline_data:
        blob = artifact_part.inline_data
        size = len(blob.data)
        mime = blob.mime_type

        return (
            f"Artifact Loaded Successfully\n"
            f"Filename: {filename}\n"
            f"MIME Type: {mime}\n"
            f"Size: {size} bytes"
        )

    return "Artifact has no inline binary data."


# ============================================================
# TOOL 3: LIST ARTIFACTS
# ============================================================

async def list_artifacts_tool(tool_context=None) -> str:

    keys = await tool_context.list_artifacts()

    if not keys:
        return "No artifacts stored."

    return "Stored Artifacts:\n" + "\n".join(keys)


# ============================================================
# WRAP TOOLS
# ============================================================

tools = [
    FunctionTool(func=save_pdf_tool),
    FunctionTool(func=load_artifact_tool),
    FunctionTool(func=list_artifacts_tool),
]

# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="ArtifactDataAgent",
    model=MODEL,
    instruction="""
You manage binary artifacts.

You can:
- Save a PDF artifact
- Load an artifact
- Show MIME type
- List stored artifacts

Always use tools when performing artifact operations.
""",
    tools=tools
)

# ============================================================
# RUNNER
# ============================================================

async def run_demo():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    print("=" * 60)
    print("Artifact Data (Binary + MIME Type) Demo Started")
    print("=" * 60)
    print("Examples:")
    print(" - Save sample.pdf")
    print(" - Load sample.pdf")
    print(" - List artifacts")
    print("Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit", ""]:
            break

        content = types.Content(
            role="user",
            parts=[types.Part(text=user_input)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("Assistant:", event.content.parts[0].text)


# ============================================================
# ENTRY POINT
# ============================================================

if __name__ == "__main__":
    # asyncio.run(run_demo())
    await run_demo()



Artifact Data (Binary + MIME Type) Demo Started
Examples:
 - Save sample.pdf
 - Load sample.pdf
 - List artifacts
Type 'exit' to quit.



You:  exit


### **<font color="blue">Filename</font>**
- **Identifier:** A simple string used to name and retrieve an artifact within its specific namespace.
- **Uniqueness:** Filenames must be unique within their scope (either the session or the user namespace).
- **Best Practice:** Use descriptive names, potentially including file extensions(e.g., `"monthly_report.pdf"`, `"user_avatar.jpg"`, although the extension itself doesn't dictate behavior-the `mime_type` does.
### **<font color="blue">Versioning</font>**
- **Automatic Versioning:** The artifact service automatically handles versioning When you call `save_artifact`, the service determines the next available version number (typically starting from 0 and incrementing) for that specific filename and scope.
- **Returned by `save_artifact`:** Then `save_artifact` method returns the integer version number that was assigned to the newly saved artifact.
- **Retrieval:**
  - **`load_artifact(..., version=None)` (default):** Retrieves the _latest_ available version of the artifact.
  - **`load_artifact(..., version=N)`:** Retrieves the specific version `N`.
- **Listing Versions:** The `list_versions` method (on the service, not context) can be used to find all existing version numbers for an artifact.
### **<font color="blue">Namespacing (Session vs. User)</font>**
- **Concept:** Artifacts can be scoped either to a specific `session` or more broadly to a user across all their **sessions** within the applicaiton. This scope is determined by the `filename` format and handled internally by the `ArtifactService`.
- **Default (Session Scope):** If you use a plain filename like `"report.pdf"`, the artifact is associated with the specific `app_name`, `user_id`, and `session_id`. It's only accessible within that exact session context.
- **User Scope (`"user:"` prefix):** If you prefix the finename with `"user:"`, like `"user:profile.png"`, the artifact is associated only with the `app_name` and `user_id`. It can be accessed or updated from _any_ session belogning to that user within the app.

In [5]:
# ============================================================
# ADK Artifact Namespace Demo (Session vs User Scope)
# Fully Corrected Version
# ============================================================

import os
import asyncio
import json

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
import google.genai.types as types
from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

MODEL = "gemini-2.0-flash"
APP_NAME = "artifact_namespace_demo"
USER_ID = "user_1"

SESSION_ID_1 = "session_A"
SESSION_ID_2 = "session_B"

# ============================================================
# TOOL 1: SAVE SESSION ARTIFACT (Session Scoped)
# ============================================================

async def save_session_artifact(tool_context=None):

    content = "This is a session-specific summary."

    artifact = types.Part.from_bytes(
        data=content.encode("utf-8"),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename="summary.txt",  # session scoped
        artifact=artifact
    )

    return f"Session artifact 'summary.txt' saved (version {version})."


# ============================================================
# TOOL 2: SAVE USER ARTIFACT (User Scoped via prefix)
# ============================================================

async def save_user_settings(tool_context=None):

    settings = {
        "theme": "dark",
        "language": "en",
        "notifications": True
    }

    json_bytes = json.dumps(settings, indent=2).encode("utf-8")

    artifact = types.Part(
        inline_data=types.Blob(
            data=json_bytes,
            mime_type="application/json"
        )
    )

    version = await tool_context.save_artifact(
        filename="user:settings.json",  # user scoped
        artifact=artifact
    )

    return f"User artifact 'user:settings.json' saved (version {version})."


# ============================================================
# TOOL 3: LOAD ARTIFACT
# ============================================================

async def load_artifact(filename: str, tool_context=None):

    artifact_part = await tool_context.load_artifact(filename=filename)

    if not artifact_part:
        return f"Artifact '{filename}' not found."

    if artifact_part.inline_data:
        data = artifact_part.inline_data.data.decode("utf-8")
        mime = artifact_part.inline_data.mime_type

        return (
            f"Artifact Loaded:\n"
            f"Filename: {filename}\n"
            f"MIME Type: {mime}\n"
            f"Content:\n{data}"
        )

    return "Artifact exists but has no inline data."


# ============================================================
# REGISTER TOOLS
# ============================================================

tools = [
    FunctionTool(func=save_session_artifact),
    FunctionTool(func=save_user_settings),
    FunctionTool(func=load_artifact),
]

# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="NamespaceAgent",
    model=MODEL,
    instruction="""
You manage artifacts with namespace scopes.

Session artifact:
- summary.txt

User artifact:
- user:settings.json

Use tools when saving or loading artifacts.
""",
    tools=tools
)

# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    # Create sessions
    ## Session 1
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID_1
    )

    ## Session 2
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID_2
    )

    # Runner
    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    # Invocation
    async def invoke(session_id, message_text):

        ## content
        content = types.Content(
            role="user",
            parts=[types.Part(text=message_text)]
        )

        ## events
        events = runner.run_async(
            user_id=USER_ID,
            session_id=session_id,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("Assistant:", event.content.parts[0].text)

    # Session 1
    print("\n=== SESSION A: Saving Artifacts ===\n")
    
    await invoke(SESSION_ID_1, "Save session artifact")
    await invoke(SESSION_ID_1, "Save user settings")
    
    print("\n=== SESSION A: Testing Visibility ===\n")
    
    ## Should see session artifact
    await invoke(SESSION_ID_1, "Load summary.txt")

    ## Should see user artifact
    await invoke(SESSION_ID_1, "Load user:settings.json")

    print("\n=== SESSION B: Testing Visibility ===\n")

    ## Should NOT see session artifact
    await invoke(SESSION_ID_2, "Load summary.txt")

    ## Should see user artifact
    await invoke(SESSION_ID_2, "Load user:settings.json")


# ============================================================
# EXECUTION
# ============================================================

# If running in Jupyter:
await run_demo()

# If running as normal Python script instead, use:
# if __name__ == "__main__":
#     asyncio.run(run_demo())



=== SESSION A: Saving Artifacts ===

Assistant: OK. I have saved the session artifact 'summary.txt'.

Assistant: OK. I have saved the user artifact 'user:settings.json'.


=== SESSION A: Testing Visibility ===

Assistant: OK. I have loaded the artifact 'summary.txt'. The content is: This is a session-specific summary.

Assistant: OK. I have loaded the artifact 'user:settings.json'. The content is: 
```json
{
  "theme": "dark",
  "language": "en",
  "notifications": true
}
```

=== SESSION B: Testing Visibility ===

Assistant: I was unable to load the artifact. It appears that the artifact 'summary.txt' does not exist. Did you want me to load a different artifact?
Assistant: OK. I have loaded the artifact 'user:settings.json'. The content is:

```
{
  "theme": "dark",
  "language": "en",
  "notifications": true
}
```



In [12]:
# ============================================================
# ADK Artifact Namespace Demo (Real World Example)
# AI Resume Builder — Session vs User Scope
# ============================================================

import os
import asyncio
import json

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
import google.genai.types as types
from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

MODEL = "gemini-2.5-flash"
APP_NAME = "ai_resume_builder"
USER_ID = "user_1"

SESSION_ID_1 = "resume_edit_session"
SESSION_ID_2 = "resume_review_session"

# ============================================================
# TOOL 1: SAVE SESSION ARTIFACT (Resume Draft)
# ============================================================

async def save_resume_draft(tool_context=None):

    content = """John Doe
Software Engineer

Experience:
- Built scalable backend systems
- Worked with Python and AI technologies
"""

    artifact = types.Part.from_bytes(
        data=content.encode("utf-8"),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename="resume_draft.txt",  # session scoped
        artifact=artifact
    )

    return f"Resume draft saved (version {version})."


# ============================================================
# TOOL 2: SAVE USER ARTIFACT (User Preferences)
# ============================================================

async def save_user_preferences(tool_context=None):

    preferences = {
        "template": "modern",
        "language": "en",
        "font": "Helvetica"
    }

    json_string = json.dumps(preferences, indent=2)

    # Use from_bytes for consistent behavior
    artifact = types.Part.from_bytes(
        data=json_string.encode("utf-8"),
        mime_type="application/json"
    )

    version = await tool_context.save_artifact(
        filename="user:preferences.json",  # user scoped
        artifact=artifact
    )

    return f"User preferences saved (version {version})."


# ============================================================
# TOOL 3: LOAD ARTIFACT
# ============================================================

async def load_artifact(filename: str, tool_context=None):

    artifact_part = await tool_context.load_artifact(filename=filename)

    if not artifact_part:
        return f"Artifact '{filename}' not found."

    # Handle both inline_data and from_bytes storage
    if artifact_part.inline_data and artifact_part.inline_data.data:
        data = artifact_part.inline_data.data.decode("utf-8")
        mime = artifact_part.inline_data.mime_type

        return (
            f"\nArtifact Loaded:\n"
            f"Filename: {filename}\n"
            f"MIME Type: {mime}\n"
            f"Content:\n{data}"
        )

    return "Artifact exists but content could not be decoded."


# ============================================================
# REGISTER TOOLS
# ============================================================

tools = [
    FunctionTool(func=save_resume_draft),
    FunctionTool(func=save_user_preferences),
    FunctionTool(func=load_artifact),
]

# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="ResumeAgent",
    model=MODEL,
    instruction="""
You are an AI Resume Assistant.

Session Artifact:
- resume_draft.txt (only available in current editing session)

User Artifact:
- user:preferences.json (available across all user sessions)

Always call tools when user asks to save or load artifacts.
""",
    tools=tools
)

# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    # Create sessions
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID_1
    )

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID_2
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(session_id, message_text):

        content = types.Content(
            role="user",
            parts=[types.Part(text=message_text)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=session_id,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("Assistant:", event.content.parts[0].text)

    # SESSION 1
    print("\n=== SESSION A: Editing Resume ===\n")

    await invoke(SESSION_ID_1, "Save resume draft")
    await invoke(SESSION_ID_1, "Save user preferences")

    print("\n=== SESSION A: Checking Visibility ===\n")

    await invoke(SESSION_ID_1, "Load resume_draft.txt")
    await invoke(SESSION_ID_1, "Load user:preferences.json")

    # SESSION 2
    print("\n=== SESSION B: Reviewing Resume ===\n")

    await invoke(SESSION_ID_2, "Load resume_draft.txt")  # Should NOT exist
    await invoke(SESSION_ID_2, "Load user:preferences.json")  # Should exist


# ============================================================
# EXECUTION
# ============================================================

await run_demo()

# For script:
# if __name__ == "__main__":
#     asyncio.run(run_demo())



=== SESSION A: Editing Resume ===

Assistant: OK. I've saved your resume draft.
Assistant: I've saved your user preferences.

=== SESSION A: Checking Visibility ===

Assistant: I've loaded your resume draft. It contains:

John Doe
Software Engineer

Experience:
- Built scalable backend systems
- Worked with Python and AI technologies
Assistant: I've loaded your user preferences. They are:



=== SESSION B: Reviewing Resume ===

Assistant: I couldn't find 'resume_draft.txt'. Are you sure it exists?
Assistant: I have loaded your user preferences. Here's the content:




## **<font color="green">Interacting with Artifacts (via Context Objects)</font>**
The primary way you interact with artifacts within your agent's logic (specifically within callbacks or tools) is through methods provided by the `CallbackContext` and `ToolContext` objects. These methods ***abstract*** away the underlying storage details managed by the `ArtifactService`.
### **<font color="blue">Prerequisite: Configuring the ArtifactService</font>**
Before we can use the artifact methods via the context objects, you _must_ provide an instance of a `BaseArtifactService` __implementation__ like (`InMemoryArtifactService` or `GcsArtifactService`) when intializing our `Runner`.

In [20]:
# ============================================================
# ADK Artifact Interaction Demo
# Dynamic Content + Versioning Across Sessions
# Flow: User → LLM → Tool → ToolContext → ArtifactService
# ============================================================

import os
import asyncio
import json
from datetime import datetime

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
import google.genai.types as types

from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "artifact_app"
USER_ID = "user_1"

SESSION_ID_1 = "session_A"
SESSION_ID_2 = "session_B"

MODEL = "gemini-2.5-flash"

# ============================================================
# TOOL 1: Save Session Artifact (Dynamic Content from User)
# ============================================================

async def save_session_file(content: str, tool_context=None):
    """
    Saves dynamic session-scoped artifact.
    Each save creates a new version.
    The content is derived from user input.
    """

    dynamic_content = (
        f"Session Notes\n"
        f"Content: {content}\n"
        f"Saved at: {datetime.utcnow()}\n"
    )

    artifact = types.Part.from_bytes(
        data=dynamic_content.encode("utf-8"),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename="notes.txt",
        artifact=artifact
    )

    return f"Session artifact saved as notes.txt (version {version})."


# ============================================================
# TOOL 2: Save User Artifact (Dynamic JSON)
# ============================================================

async def save_user_profile(tool_context=None):

    profile = {
        "name": "John Doe",
        "subscription": "pro",
        "last_updated": str(datetime.utcnow())
    }

    json_string = json.dumps(profile, indent=2)

    artifact = types.Part.from_bytes(
        data=json_string.encode("utf-8"),
        mime_type="application/json"
    )

    version = await tool_context.save_artifact(
        filename="user:profile.json",
        artifact=artifact
    )

    return f"User artifact saved as user:profile.json (version {version})."


# ============================================================
# TOOL 3: Load Artifact (Latest Version)
# ============================================================

async def load_file(filename: str, tool_context=None):

    artifact_part = await tool_context.load_artifact(filename=filename)

    if not artifact_part:
        return f"Artifact '{filename}' not found."

    if artifact_part.inline_data and artifact_part.inline_data.data:
        data = artifact_part.inline_data.data.decode("utf-8")
        mime = artifact_part.inline_data.mime_type

        return (
            f"\nArtifact Loaded\n"
            f"Filename: {filename}\n"
            f"MIME Type: {mime}\n"
            f"Content:\n{data}"
        )

    return "Artifact exists but no readable content."


# ============================================================
# REGISTER TOOLS
# ============================================================

tools = [
    FunctionTool(func=save_session_file),
    FunctionTool(func=save_user_profile),
    FunctionTool(func=load_file),
]

# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="ArtifactAgent",
    model=MODEL,
    instruction="""
You manage artifacts.

Session file:
- notes.txt

User file:
- user:profile.json

When user says:
"Save session file for X"
Extract X and pass it as content to save_session_file.

Always use tools when saving or loading files.
""",
    tools=tools
)

# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    # Create 2 sessions
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID_1
    )

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID_2
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(session_id, message):

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=session_id,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("Assistant:", event.content.parts[0].text)

    # =========================================================
    # SESSION A - Create Multiple Versions
    # =========================================================

    print("\n=== SESSION A: Creating Versions ===\n")

    await invoke(SESSION_ID_1, "Save session file for Red")    # version 0
    await asyncio.sleep(1)

    await invoke(SESSION_ID_1, "Save session file for Blue")   # version 1
    await asyncio.sleep(1)

    await invoke(SESSION_ID_1, "Save session file for Green")  # version 2
    await asyncio.sleep(1)

    await invoke(SESSION_ID_1, "Save user preferences") # user preferences

    print("\n=== Load Latest Version in SESSIONS ===\n")
    print("=== SESSION A ===\n")
    print("-"*80)
    await invoke(SESSION_ID_1, "Load notes.txt") # Load latest version
    print("-"*80)
    print("-"*80)
    await invoke(SESSION_ID_1, "Load user:profile.json") # Load user
    print("-"*80)

    # =========================================================
    # SESSION B - New Session
    # =========================================================

    print("\n=== SESSION B ===\n")

    await invoke(SESSION_ID_2, "Load notes.txt")  # Should NOT exist

    print("\n--- Save New Version in SESSION B ---\n")
    await invoke(SESSION_ID_2, "Save session file for Black")  # version 0
    await asyncio.sleep(1)

    print("\n--- Load SESSION B Version ---\n")
    print("-"*80)
    await invoke(SESSION_ID_2, "Load notes.txt")
    print("-"*80)
    print("-"*80)
    await invoke(SESSION_ID_1, "Load user:profile.json") # Load user
    print("-"*80)


# ============================================================
# EXECUTION
# ============================================================

await run_demo()

# For script:
# if __name__ == "__main__":
#     asyncio.run(run_demo())



=== SESSION A: Creating Versions ===

Assistant: Session file saved for Red.
Assistant: Session file saved for Blue.
Assistant: Session file saved for Green.
Assistant: User preferences saved.

=== Load Latest Version in SESSIONS ===

=== SESSION A ===

--------------------------------------------------------------------------------
Assistant: Here's the content of notes.txt:

Session Notes
Content: Green
Saved at: 2026-02-19 07:29:56.819761
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Assistant: Here's the content of user:profile.json:

{
  "name": "John Doe",
  "subscription": "pro",
  "last_updated": "2026-02-19 07:30:01.224321"
}
--------------------------------------------------------------------------------

=== SESSION B ===

Assistant: Artifact 'notes.txt' not found.

--- Save New Version in SESSION B ---

Assistant: Session artifact saved as notes.txt (version 

In [24]:
# ============================================================
# ADK Callback + CallbackContext Artifact Demo
# ============================================================

import os
import asyncio
from datetime import datetime

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
import google.genai.types as types

from config import config

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "callback_artifact_app"
USER_ID = "user_1"
SESSION_ID = "session_callback"
MODEL = "gemini-2.5-flash"


# ============================================================
# CALLBACK (Corrected)
# ============================================================

async def after_model_callback(
    callback_context,
    llm_response=None,
    **kwargs
):
    """
    Runs AFTER the LLM generates a response.
    Saves the LLM output as an artifact.
    """

    # Extract text safely
    if llm_response and llm_response.content and llm_response.content.parts:
        response_text = llm_response.content.parts[0].text
    else:
        response_text = "No response text available."

    artifact_content = (
        f"LLM Response Log\n"
        f"Time: {datetime.utcnow()}\n\n"
        f"{response_text}"
    )

    artifact_part = types.Part.from_bytes(
        data=artifact_content.encode("utf-8"),
        mime_type="text/plain"
    )

    version = await callback_context.save_artifact(
        filename="llm_response.txt",
        artifact=artifact_part
    )

    print(f"\n[Callback] Saved llm_response.txt (version {version})")


# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="CallbackAgent",
    model=MODEL,
    instruction="Answer clearly and concisely.",
    after_model_callback=after_model_callback
)


# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(message):

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("\nAssistant:", event.content.parts[0].text)

    print("\n=== CALLBACK FLOW DEMO ===")

    await invoke("What is artificial intelligence?")
    await asyncio.sleep(1)

    await invoke("Explain machine learning in one sentence.")
    await asyncio.sleep(1)

    print("\n=== Loading Latest Saved Artifact ===\n")

    artifact_part = await artifact_service.load_artifact(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        filename="llm_response.txt"
    )

    if artifact_part and artifact_part.inline_data:
        print(artifact_part.inline_data.data.decode("utf-8"))
    else:
        print("No artifact found.")


# For Jupyter
await run_demo()

# For script
# if __name__ == "__main__":
#     asyncio.run(run_demo())



=== CALLBACK FLOW DEMO ===

[Callback] Saved llm_response.txt (version 0)

Assistant: Artificial intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, problem-solving, perception, and understanding language.

[Callback] Saved llm_response.txt (version 1)

Assistant: Machine learning is a subset of AI that enables systems to learn and improve from experience (data) without being explicitly programmed.

=== Loading Latest Saved Artifact ===

LLM Response Log
Time: 2026-02-19 08:30:57.420016

Machine learning is a subset of AI that enables systems to learn and improve from experience (data) without being explicitly programmed.


### **<font color="blue">Accessing Methods</font>**
The artifact interaction methods are available directly on instances of `CallbackContext` (passed to agent and model callbacks) and `ToolContext` (passed to tool callbacks). Remember that `ToolContext` inherits from `CallbackContext`.
-  Saving Artifacts
-  Loading Artifacts
-  Listing Artifacts Filenames

In [32]:
# ============================================================
# ADK Artifact Demo
# Save 3 Artifacts → List → Load All One by One
# ============================================================

import os
import asyncio
from datetime import datetime

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types

from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "artifact_multi_demo"
USER_ID = "user_1"
SESSION_ID = "session_1"
MODEL = "gemini-2.5-flash"


# ============================================================
# 1️⃣ SAVE ARTIFACT
# ============================================================

async def save_artifacts(
    filename: str,
    content: str,
    tool_context: ToolContext = None
):

    artifact_content = (
        f"Saved at: {datetime.utcnow()}\n\n"
        f"{content}"
    )

    artifact_part = types.Part.from_bytes(
        data=artifact_content.encode("utf-8"),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=artifact_part
    )

    return f"✅ '{filename}' saved (version {version})"


# ============================================================
# 2️⃣ LOAD ARTIFACT
# ============================================================

async def load_artifacts(
    filename: str,
    tool_context: ToolContext = None
):

    artifact = await tool_context.load_artifact(filename=filename)

    if not artifact or not artifact.inline_data:
        return f"'{filename}' not found."

    data = artifact.inline_data.data.decode("utf-8")

    return (
        f"\nContent of '{filename}':\n"
        f"{'-'*40}\n"
        f"{data}\n"
        f"{'-'*40}"
    )


# ============================================================
# 3️⃣ LIST ARTIFACTS
# ============================================================

async def list_artifacts(
    tool_context: ToolContext = None
):

    files = await tool_context.list_artifacts()

    if not files:
        return "No artifacts found."

    formatted = "\n".join([f"- {f}" for f in files])

    return f"\nAvailable Artifacts:\n{formatted}"


# ============================================================
# REGISTER TOOLS
# ============================================================

tools = [
    FunctionTool(func=save_artifacts),
    FunctionTool(func=load_artifacts),
    FunctionTool(func=list_artifacts),
]


# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="ArtifactToolAgent",
    model=MODEL,
    instruction="""
You are a helpful assistant.

When user says:
- Save <filename> with <content> → call save_artifacts
- Load <filename> → call load_artifacts
- List files → call list_artifacts

Always use the appropriate tool.
""",
    tools=tools
)


# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(message):

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("\nAssistant:", event.content.parts[0].text)

    print("\n=== MULTI ARTIFACT DEMO ===\n")

    # --------------------------------------------------------
    # 1️⃣ Save Three Artifacts
    # --------------------------------------------------------

    await invoke("Save file1.txt with This is the first artifact.")
    await asyncio.sleep(1)

    await invoke("Save file2.txt with This is the second artifact.")
    await asyncio.sleep(1)

    await invoke("Save file3.txt with This is the third artifact.")
    await asyncio.sleep(1)

    await invoke("Save secreat.txt with This is the confidential artifact file.")
    await asyncio.sleep(1)

    # --------------------------------------------------------
    # 2️⃣ List All Artifacts
    # --------------------------------------------------------

    await invoke("List files")
    await asyncio.sleep(1)

    # --------------------------------------------------------
    # 3️⃣ Load All Three One by One
    # --------------------------------------------------------

    await invoke("Load file1.txt")
    await asyncio.sleep(1)

    await invoke("Load file2.txt")
    await asyncio.sleep(1)

    await invoke("Load file3.txt")
    await asyncio.sleep(1)
    
    await invoke("Load secreat.txt")
    await asyncio.sleep(1)

# ============================================================
# EXECUTION
# ============================================================

# For Jupyter
await run_demo()

# For script
# if __name__ == "__main__":
#     asyncio.run(run_demo())



=== MULTI ARTIFACT DEMO ===


Assistant: I have saved the content to `file1.txt`.

Assistant: I have saved the content to `file2.txt`.

Assistant: I have saved the content to `file3.txt`.

Assistant: I have saved the content to `secreat.txt`.

Assistant: Here is a list of your files:
- file1.txt
- file2.txt
- file3.txt
- secreat.txt

Assistant: Here is the content of `file1.txt`:


Assistant: Here is the content of `file2.txt`:


Assistant: Here is the content of `file3.txt`:


Assistant: Here is the content of `secreat.txt`:



In [33]:
# ============================================================
# ADK Artifact Full Demo (3 Auto-Saved Artifacts Version)
# - Save 3 artifacts via CallbackContext
# - Load via ToolContext
# - List via ToolContext
# ============================================================

import os
import asyncio
from datetime import datetime

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.agents.callback_context import CallbackContext
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types

from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "artifact_full_demo"
USER_ID = "user_1"
SESSION_ID = "session_1"
MODEL = "gemini-2.5-flash"

# Global counter for file rotation
artifact_counter = 0


# ============================================================
# CALLBACK: Save 3 Separate Artifacts
# ============================================================

async def after_model_callback(
    callback_context: CallbackContext,
    llm_response=None,
    **kwargs
):

    global artifact_counter

    if not llm_response or not llm_response.content:
        return

    parts = llm_response.content.parts or []

    texts = [
        part.text for part in parts
        if hasattr(part, "text") and part.text
    ]

    if not texts:
        return

    response_text = "\n".join(texts)

    artifact_content = (
        f"LLM Auto Log\n"
        f"Time: {datetime.utcnow()}\n\n"
        f"{response_text}"
    )

    artifact_part = types.Part.from_bytes(
        data=artifact_content.encode("utf-8"),
        mime_type="text/plain"
    )

    # Rotate filenames for first 3 responses
    artifact_counter += 1
    filename = f"llm_log_{artifact_counter}.txt"

    await callback_context.save_artifact(
        filename=filename,
        artifact=artifact_part
    )

    print(f"\n[Callback] Saved {filename}")


# ============================================================
# TOOL: Load Artifact
# ============================================================

async def load_artifact_file(
    filename: str,
    tool_context: ToolContext = None
):

    artifact = await tool_context.load_artifact(filename=filename)

    if not artifact or not artifact.inline_data:
        return f"Artifact '{filename}' not found."

    data = artifact.inline_data.data.decode("utf-8")

    return (
        f"\nLoaded: {filename}\n"
        f"{'-'*40}\n"
        f"{data}\n"
        f"{'-'*40}"
    )


# ============================================================
# TOOL: List Artifacts
# ============================================================

async def list_artifacts(tool_context: ToolContext = None):

    files = await tool_context.list_artifacts()

    if not files:
        return "No artifacts found."

    formatted = "\n".join([f"- {f}" for f in files])
    return f"\nAvailable Artifacts:\n{formatted}"


# ============================================================
# REGISTER TOOLS
# ============================================================

tools = [
    FunctionTool(func=load_artifact_file),
    FunctionTool(func=list_artifacts),
]


# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="ArtifactFullAgent",
    model=MODEL,
    instruction="""
You are a helpful assistant.

If user says:
- "Load <filename>" → use load_artifact_file
- "List files" → use list_artifacts

All meaningful responses are automatically saved
into separate artifact files.
""",
    tools=tools,
    after_model_callback=after_model_callback
)


# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(message):

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("\nAssistant:", event.content.parts[0].text)

    print("\n=== ARTIFACT FULL DEMO (3 FILE VERSION) ===\n")

    # --------------------------------------------------------
    # 1️⃣ Generate 3 Responses (Auto-saved separately)
    # --------------------------------------------------------

    await invoke("What is cloud computing?")
    await asyncio.sleep(1)

    await invoke("Explain APIs briefly.")
    await asyncio.sleep(1)

    await invoke("What is artificial intelligence?")
    await asyncio.sleep(1)

    # --------------------------------------------------------
    # 2️⃣ List All Artifacts
    # --------------------------------------------------------

    await invoke("List files")
    await asyncio.sleep(1)

    # --------------------------------------------------------
    # 3️⃣ Load All Three One by One
    # --------------------------------------------------------

    await invoke("Load llm_log_1.txt")
    await asyncio.sleep(1)

    await invoke("Load llm_log_2.txt")
    await asyncio.sleep(1)

    await invoke("Load llm_log_3.txt")


# ============================================================
# EXECUTION
# ============================================================

# For Jupyter
await run_demo()

# For script
# if __name__ == "__main__":
#     asyncio.run(run_demo())



=== ARTIFACT FULL DEMO (3 FILE VERSION) ===


[Callback] Saved llm_log_1.txt

Assistant: Cloud computing is an on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining your own compute data centers and servers, you can access technology services, such as computing power, storage, and databases, from a cloud provider like Amazon Web Services (AWS), Google Cloud, or Microsoft Azure.

[Callback] Saved llm_log_2.txt

Assistant: An Application Programming Interface (API) is a set of rules and protocols that allows different software applications to communicate and interact with each other. It defines the methods and data formats that applications can use to request and exchange information.

[Callback] Saved llm_log_3.txt

Assistant: Artificial intelligence (AI) is a broad field of computer science that gives computers the ability to perform human-like cognitive functions such as learning, problem-solving, and decision-maki

## **<font color="green">Available Implementations</font>**
### **<font color="blue">InMemoryArtifactService</font>**
- **Storage Mechanism**
  - Python: Uses a python dictionary (`self.artifacts`) held in the applications memory. The dictionary keys represent the artifact path, and the value are lists of `types.Part`, where each list element is a version.
- **Key Features**
  - ___Simplicity: Requires:___ Requires no external setup or dependencies beyond the core ADK library.
  - ___Speed:___ Operations are typically very fast as they involve in-memory map/dictionary lookups and list manupulations.
  - ___Ephemeral:___ All stored artifacts are **lost** when the application process terminates. Data does not persist between application restarts.
- **Use Cases**:
  - Ideal for local development and testing where persistence is not required.
  - Suitable for short-lived demonstrations or scenarios where artifact data is purely temporary within a single run of the applicaiton.
- **Installation**

In [39]:
# ============================================================
# Simple InMemoryArtifactService Example (Correct & Minimal)
# ============================================================

import asyncio
from google.adk.artifacts import InMemoryArtifactService
import google.genai.types as types


async def main():

    artifact_service = InMemoryArtifactService()

    app_name = "demo_app"
    user_id = "user_1"
    session_id = "session_1"

    # Create content
    content = "Hello, this is a test artifact."

    artifact_part = types.Part.from_bytes(
        data=content.encode("utf-8"),
        mime_type="text/plain"
    )

    # Save artifact
    version = await artifact_service.save_artifact(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
        filename="sample.txt",
        artifact=artifact_part
    )

    print("Saved version:", version)

    # Load artifact
    loaded = await artifact_service.load_artifact(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
        filename="sample.txt"
    )

    data = loaded.inline_data.data.decode("utf-8")
    print("Loaded content:", data)


# Run
await main()  # Jupyter
# asyncio.run(main())  # Script


Saved version: 0
Loaded content: Hello, this is a test artifact.


### **<font color="blue">GcsArtifactService</font>**
-  **Storage Mechanism:** Leverages _Google Cloud Storage_ (GCP) for persistent artifact storage. Each version of the artifact is stored as a separate object (blob) within a specified GCS bucket.
-  **Object Naming Convention:**  It constructs GCS object names(blob names) using a hierarchical path structure.
-  **Key Features:**
   - ___Persistence:___ Artifacts stored in GCS persist across application restarts and deployments
   - ___Scalability:___ Leverages the scalability and durability of Google Cloud Storage.
   - ___Versioning:___ Explicitly stores each version as a distinct GCS object. The `saveArtifact` method in `GcsArtifactService`.
   - ___Permissions Required:___ The application environment needs appropriate credentials(e.g., Application Default Credentials) and IAM permissions to read from and write to the specified GCS bucket.
-  **Use Cases:**
   - Production environments requiring persistent artifact storage.
   - Scenarios where artifacts need to be shared across different application instances or services (by accessing the same GCS bucket).
   - Applications needing long-term storage and retrieval of user or session data.
-  **Installation:**

In [1]:
import asyncio
from google.adk.artifacts import GcsArtifactService
import google.genai.types as types
from config import config


async def main():
    # Initialize Artifact Service
    artifact_service = GcsArtifactService(
        bucket_name=config.BUCKET_NAME
    )

    # Identifiers
    app_name = "demo_app"
    user_id = "user_1"
    session_id = "session_1"

    # Content to save
    content = "Hello from GCS Artifact Service!"

    artifact_part = types.Part.from_bytes(
        data=content.encode("utf-8"),
        mime_type="text/plain"
    )

    # ------------------------
    # SAVE ARTIFACT
    # ------------------------
    version = await artifact_service.save_artifact(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
        filename="sample.txt",
        artifact=artifact_part
    )

    print("Saved version:", version)

    # ------------------------
    # LOAD ARTIFACT
    # ------------------------
    loaded = await artifact_service.load_artifact(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
        filename="sample.txt"
    )

    content_loaded = loaded.inline_data.data.decode("utf-8")
    print("Loaded content:", content_loaded)


# ------------------------
# Execution Block
# ------------------------

await main() # Jupyter-Lab

# if __name__ == "__main__":
#     asyncio.run(main())



Saved version: 1
Loaded content: Hello from GCS Artifact Service!


In [2]:
# ============================================================
# ADK Artifact Demo (GCS Version)
# Save 3 Artifacts → List → Load All One by One
# ============================================================

import os
import asyncio
from datetime import datetime

from google.adk.runners import Runner
from google.adk.artifacts import GcsArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types

from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "artifact_multi_demo"
USER_ID = "user_1"
SESSION_ID = "session_1"
MODEL = "gemini-2.5-flash"


# ============================================================
# 1️⃣ SAVE ARTIFACT
# ============================================================

async def save_artifacts(
    filename: str,
    content: str,
    tool_context: ToolContext = None
):
    artifact_content = (
        f"Saved at: {datetime.utcnow()}\n\n"
        f"{content}"
    )

    artifact_part = types.Part.from_bytes(
        data=artifact_content.encode("utf-8"),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=artifact_part
    )

    return f"✅ '{filename}' saved (version {version})"


# ============================================================
# 2️⃣ LOAD ARTIFACT
# ============================================================

async def load_artifacts(
    filename: str,
    tool_context: ToolContext = None
):
    artifact = await tool_context.load_artifact(filename=filename)

    if not artifact or not artifact.inline_data:
        return f"'{filename}' not found."

    data = artifact.inline_data.data.decode("utf-8")

    return (
        f"\nContent of '{filename}':\n"
        f"{'-'*40}\n"
        f"{data}\n"
        f"{'-'*40}"
    )


# ============================================================
# 3️⃣ LIST ARTIFACTS
# ============================================================

async def list_artifacts(
    tool_context: ToolContext = None
):
    files = await tool_context.list_artifacts()

    if not files:
        return "No artifacts found."

    formatted = "\n".join([f"- {f}" for f in files])

    return f"\nAvailable Artifacts:\n{formatted}"


# ============================================================
# REGISTER TOOLS
# ============================================================

tools = [
    FunctionTool(func=save_artifacts),
    FunctionTool(func=load_artifacts),
    FunctionTool(func=list_artifacts),
]


# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="ArtifactToolAgent",
    model=MODEL,
    instruction="""
You are a helpful assistant.

When user says:
- Save <filename> with <content> → call save_artifacts
- Load <filename> → call load_artifacts
- List files → call list_artifacts

Always use the appropriate tool.
""",
    tools=tools
)


# ============================================================
# RUN DEMO (NOW USING GCS)
# ============================================================

async def run_demo():

    # 🔥 CHANGED HERE → GCS Artifact Service
    artifact_service = GcsArtifactService(
        bucket_name=config.BUCKET_NAME
    )

    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(message):
        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("\nAssistant:", event.content.parts[0].text)

    print("\n=== GCS MULTI ARTIFACT DEMO ===\n")

    # Save 4 artifacts
    await invoke("Save file1.txt with This is the first artifact.")
    await asyncio.sleep(1)

    await invoke("Save file2.txt with This is the second artifact.")
    await asyncio.sleep(1)

    await invoke("Save file3.txt with This is the third artifact.")
    await asyncio.sleep(1)

    await invoke("Save secreat.txt with This is the confidential artifact file.")
    await asyncio.sleep(1)

    # List
    await invoke("List files")
    await asyncio.sleep(1)

    # Load one by one
    await invoke("Load file1.txt")
    await asyncio.sleep(1)

    await invoke("Load file2.txt")
    await asyncio.sleep(1)

    await invoke("Load file3.txt")
    await asyncio.sleep(1)

    await invoke("Load secreat.txt")
    await asyncio.sleep(1)


# ============================================================
# EXECUTION
# ============================================================

await run_demo()  # For Jupyter

# For script mode:
# if __name__ == "__main__":
#     asyncio.run(run_demo())



=== GCS MULTI ARTIFACT DEMO ===



D:\Agent-Development-Kit\venv\Lib\site-packages\google\adk\flows\llm_flows\base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:



Assistant: "file1.txt" has been saved.

Assistant: "file2.txt" has been saved.

Assistant: "file3.txt" has been saved.

Assistant: "secreat.txt" has been saved.

Assistant: The following files are available:
- file1.txt
- file2.txt
- file3.txt
- secreat.txt

Assistant: Content of 'file1.txt':
----------------------------------------
Saved at: 2026-02-19 12:38:06.012881

This is the first artifact.
----------------------------------------

Assistant: Content of 'file2.txt':
----------------------------------------
Saved at: 2026-02-19 12:38:13.210014

This is the second artifact.
----------------------------------------

Assistant: Content of 'file3.txt':
----------------------------------------
Saved at: 2026-02-19 12:38:19.340382

This is the third artifact.
----------------------------------------

Assistant: Content of 'secreat.txt':
----------------------------------------
Saved at: 2026-02-19 12:38:25.913681

This is the confidential artifact file.
-------------------------------

In [5]:
# ============================================================
# Load All Artifacts from GCS (Correct Version)
# ============================================================

import asyncio
from google.cloud import storage
from google.adk.artifacts import GcsArtifactService
from config import config


APP_NAME = "artifact_multi_demo"
USER_ID = "user_1"
SESSION_ID = "session_1"


async def main():

    artifact_service = GcsArtifactService(
        bucket_name=config.BUCKET_NAME
    )

    storage_client = storage.Client()
    bucket = storage_client.bucket(config.BUCKET_NAME)

    prefix = f"{APP_NAME}/{USER_ID}/{SESSION_ID}/"

    print("\nFetching artifacts from GCS...\n")

    blobs = bucket.list_blobs(prefix=prefix)

    filenames = set()

    for blob in blobs:
        parts = blob.name.split("/")
        if len(parts) >= 4:
            filenames.add(parts[3])

    if not filenames:
        print("No artifacts found.")
        return

    print("Available Artifacts:")
    for name in filenames:
        print("-", name)

    print("\n" + "="*60)

    # Load each file using ADK service
    for filename in filenames:

        artifact = await artifact_service.load_artifact(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID,
            filename=filename
        )

        if not artifact or not artifact.inline_data:
            print(f"\n❌ Could not load: {filename}")
            continue

        content = artifact.inline_data.data.decode("utf-8")

        print(f"\n📄 Content of '{filename}':")
        print("-" * 40)
        print(content)
        print("-" * 40)

    print("\nDone.\n")


# For Jupyter
await main()

# For script:
# if __name__ == "__main__":
#     asyncio.run(main())



Fetching artifacts from GCS...

Available Artifacts:
- file1.txt
- secreat.txt
- file3.txt
- file2.txt


📄 Content of 'file1.txt':
----------------------------------------
Saved at: 2026-02-19 12:38:06.012881

This is the first artifact.
----------------------------------------

📄 Content of 'secreat.txt':
----------------------------------------
Saved at: 2026-02-19 12:38:25.913681

This is the confidential artifact file.
----------------------------------------

📄 Content of 'file3.txt':
----------------------------------------
Saved at: 2026-02-19 12:38:19.340382

This is the third artifact.
----------------------------------------

📄 Content of 'file2.txt':
----------------------------------------
Saved at: 2026-02-19 12:38:13.210014

This is the second artifact.
----------------------------------------

Done.

